<a href="https://colab.research.google.com/github/VinKKAP/Data-Analysis-with-LLM/blob/main/TryTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

# Upgrade pip und installiere die benötigten Pakete
%pip install --upgrade pip
%pip install --upgrade torch transformers
import torch

from google.colab import userdata
userdata.get('HF_TOKEN')

# Überprüfen, ob CUDA verfügbar ist, und den Namen des CUDA-Geräts abrufen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
device

device(type='cuda')

GPU ist verbunden für die berechnung, aber die

1.   Listeneintrag
2.   Listeneintrag

Instruction für das Modell scheitert daran das dass jeweilige Modell nicht richtig im Code aus dem Buch funktioniert.

In [19]:
import sqlite3

In [23]:
conn = sqlite3.connect(':memory:')

version = sqlite3.version
print(version)


2.6.0


In [61]:

import argparse
import torch
import re
import time
from transformers import pipeline

def create_prompt(question):  #1
    """ Generate prompt to translate question into SQL query.

    Args:
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += ['create table games(rank int, name text, platform text,']
    parts += ['year int, genre text, publisher text, americasales numeric,']
    parts += ['eusales numeric, japansales numeric, othersales numeric,']
    parts += ['globalsales numeric);']
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\n'.join(parts)

def call_llm(prompt):  #2
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=600, device=device, truncation=True)
    for nr_retries in range(1, 4):
        try:
            response = pipe(prompt, max_length=100)
            return response[0]['generated_text']
        except Exception as e:
            print(f"Error: {e}")
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query model!')

if __name__ == '__main__':  #3
    parser = argparse.ArgumentParser()
    parser.add_argument('question', type=str, help='A question about games')
    args = parser.parse_args(['What are the most sold games?'])

    prompt = create_prompt(args.question)
    answer = call_llm(prompt) #4
    query = re.findall(r'SELECT.*?;', answer, re.DOTALL)
    if query:
        print(f'SQL: {query[0]}')
    else:
        print("No SQL query found in the response.")

SQL: SELECT name FROM Games ORDER BY globalsales DESC LIMIT 10;


In [62]:
print(prompt)

Database:
create table games(rank int, name text, platform text,
year int, genre text, publisher text, americasales numeric,
eusales numeric, japansales numeric, othersales numeric,
globalsales numeric);
Translate this question into SQL query:
What are the most sold games?


In [63]:
print(answer)

Database:
create table games(rank int, name text, platform text,
year int, genre text, publisher text, americasales numeric,
eusales numeric, japansales numeric, othersales numeric,
globalsales numeric);
Translate this question into SQL query:
What are the most sold games? SELECT name FROM Games ORDER BY globalsales DESC LIMIT 10;
Here is a possible SQL query to answer your question:

```sql
SELECT name 
FROM Games 
ORDER BY globalsales DESC 
LIMIT


In [64]:
from transformers import pipeline

messages = [

            {"role": "user", "content": prompt},

           ]

pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=900, device='cuda')

pipe(messages)

[{'generated_text': [{'role': 'user',
    'content': 'Database:\ncreate table games(rank int, name text, platform text,\nyear int, genre text, publisher text, americasales numeric,\neusales numeric, japansales numeric, othersales numeric,\nglobalsales numeric);\nTranslate this question into SQL query:\nWhat are the most sold games?'},
   {'role': 'assistant',
    'content': 'SELECT rank, name, platform, year, genre, publisher, americasales, eusales, japansales, othersales, globalsales \nFROM games\nORDER BY globalsales DESC LIMIT 1;'}]}]

In [54]:
conn = sqlite3.connect('games.db')
